In [3]:
#Mark Hoyt 
#12/10/22

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import chain

cost_of_living = pd.read_csv('Python Project/cost_of_living.csv')
salaries = pd.read_csv('Python Project/ds_salaries.csv')
salarylevels = pd.read_csv('Python Project/Levels_Fyi_Salary_Data.csv')
country_codes = pd.read_csv('Python Project/country_codes.csv')


# making the dataframe more suitable for what I need if for 
# (exp. remote work doesn't matter for this probelm)

salaries.dtypes

#removing unneeded columns
salaries = salaries.drop(['Unnamed: 0','remote_ratio','company_size'],axis=1)
salaries["salary"] = salaries['salary'].astype(str) +" "+ salaries["salary_currency"]
salaries = salaries.drop(['salary_currency'],axis=1)

#making experience level easier to read
salaries['experience_level'] = salaries['experience_level'].replace(['MI'], 'Mid Level')
salaries['experience_level'] = salaries['experience_level'].replace(['EN'], 'Entry Level')
salaries['experience_level'] = salaries['experience_level'].replace(['EX'], 'Executive Level')
salaries['experience_level'] = salaries['experience_level'].replace(['SE'], 'Senior Level')

#making employment type easier to read 
salaries['employment_type'] = salaries['employment_type'].replace(['FT'], 'Full Time')
salaries['employment_type'] = salaries['employment_type'].replace(['FL'], 'Full Time')
salaries['employment_type'] = salaries['employment_type'].replace(['CT'], 'Contract Time')
salaries['employment_type'] = salaries['employment_type'].replace(['PT'], 'Part Time')

#coverting the date
salaries['work_year'] = pd.to_datetime(salaries['work_year'], format='%Y')

#adding a new column of country codes to country names 

salarylevels.dtypes
#removing unneeded columns
salarylevels = salarylevels.drop(['level', 'rowNumber', 'gender','tag', 'Bachelors_Degree', 'Highschool', 'Some_College', 'Race_Asian', 'Race_White', 'Race_Black', 'Race_Two_Or_More', 'Race_Hispanic', 'Race', 'Education', 'yearsatcompany','dmaid'],axis=1)

# converting time stamp to proper type
salarylevels['timestamp'] = pd.to_datetime(salarylevels['timestamp'], format='%m/%d/%Y %H:%M:%S')

#getting rid or unapplicable jobs
names=['Software Engineer','Software Engineer Manager', 'Technical Program Manager', 'Sales', 'Marketing', 'Human Resources', 'Mechanical Engineer', 'Product Designer', 'Product Manager', 'Solution Architect', 'Hardware Engineer', 'Software Engineering Manager', 'Recruiter', 'Management Consultant', 'Business Analyst']
salarylevels = salarylevels[~salarylevels.title.isin(names)]
salarylevels = salarylevels.drop(523)

# splitting location column of salarylevels
location = salarylevels["location"].str.split(",", n = 1, expand = True)
salarylevels["City/region"]= location[0]
salarylevels["State/Country"]= location[1]

#removing unneeded column 
salarylevels = salarylevels.drop(['location'],axis=1)

#moving to first index
first_column = salarylevels.pop('State/Country')
salarylevels.insert(1, 'State/Country', first_column)

cost_of_living.dtypes

# split value  location columns
location2 = cost_of_living["City"].str.split(",", n = 1, expand = True)
cost_of_living["City/region"]= location2[0]
cost_of_living["State/Country"]= location2[1]

#removing unneeded column 
cost_of_living = cost_of_living.drop(['City'],axis=1)

# creating a df in my desired index
desiredindex = cost_of_living.loc[(cost_of_living["Cost of Living Plus Rent Index"] > 50) & (cost_of_living["Cost of Living Plus Rent Index"] < 100.99) & (cost_of_living["Groceries Index"] < 100.99) & (cost_of_living["Local Purchasing Power Index"] > 99.99)]

# graph for desired index 
sns.set_theme(style="darkgrid")
sns.relplot(
    data=desiredindex, x="Cost of Living Plus Rent Index", y="Local Purchasing Power Index", hue="State/Country",
)

# I am now going to take the top purchasing power and lowest COL + rent index countrys and make a new dataframe
topindex = desiredindex.loc[(desiredindex["Local Purchasing Power Index"] > 140) & (desiredindex["Cost of Living Plus Rent Index"] < 80)]

location3 = topindex["State/Country"].str.split(",", n = 1, expand = True)
topindex["State"]= location3[0]
topindex["Country"]= location3[1]

#removing unneeded column 
topindex = topindex.drop(['State/Country'],axis=1)

#cross refrencing my top indexes locations with salarylevels locations to find matching jobs
topindex_salary = salarylevels.loc[(salarylevels["State/Country"] == ' CA') & (salarylevels["City/region"] == 'Fremont')]
topindex_salary2 = salarylevels.loc[(salarylevels["State/Country"] == ' CA') & (salarylevels["City/region"] == 'San Jose')]
topindex_salary3 = salarylevels.loc[(salarylevels["State/Country"] == ' FL')]
topindex_salary4 = salarylevels.loc[(salarylevels["State/Country"] == ' MI') & (salarylevels["City/region"] == 'Ann Arbor')]
topindex_salary5 = salarylevels.loc[(salarylevels["State/Country"] == ' NC') & (salarylevels["City/region"] == 'Charlotte')]
topindex_salary6 = salarylevels.loc[(salarylevels["State/Country"] == ' NC') & (salarylevels["City/region"] == 'Raleigh')]
topindex_salary7 = salarylevels.loc[(salarylevels["State/Country"] == ' NJ') & (salarylevels["City/region"] == 'Jersey City')]
topindex_salary8 = salarylevels.loc[(salarylevels["State/Country"] == ' OH') & (salarylevels["City/region"] == 'Columbus')]
topindex_salary9 = salarylevels.loc[(salarylevels["State/Country"] == ' TX') & (salarylevels["City/region"] == 'Dallas')]
topindex_salary10 = salarylevels.loc[(salarylevels["State/Country"] == ' TX') & (salarylevels["City/region"] == 'Austin')]
topindex_salary11 = salarylevels.loc[(salarylevels["State/Country"] == ' TX') & (salarylevels["City/region"] == 'Houston')]
topindex_salary12 = salarylevels.loc[(salarylevels["State/Country"] == ' UT')]
topindex_salary13 = salarylevels.loc[(salarylevels["State/Country"] == ' WA') & (salarylevels["City/region"] == 'Seattle')]

frame = [topindex_salary, topindex_salary2, topindex_salary3, topindex_salary4, topindex_salary5, topindex_salary6, topindex_salary7, topindex_salary8, topindex_salary9, topindex_salary10, topindex_salary11, topindex_salary12, topindex_salary13]
crossrefrence = pd.concat(frame)

# I do not have a doctoare so getting rid of this
crossrefrence = crossrefrence.loc[crossrefrence["Doctorate_Degree"] == 0]
crossrefrence.rename(columns = {'timestamp':'Job Date'}, inplace = True)

#combining state and country columns 
crossrefrence["Location"] = crossrefrence['City/region'].astype(str) +","+ crossrefrence["State/Country"]

crossrefrence = crossrefrence.drop(['State/Country'],axis=1)
crossrefrence = crossrefrence.drop(['City/region'],axis=1)

#moving to first index
first_column = crossrefrence.pop('Location')
crossrefrence.insert(1, 'Location', first_column)

#graphing crossrefrence df
sns.set_theme(style="darkgrid")
sns.relplot(
    data=crossrefrence, x="Job Date", y="totalyearlycompensation", hue="Location",
)

#showing stats of each location
avgsalaries = crossrefrence.groupby('Location')['totalyearlycompensation'].agg([np.mean, np.median, np.std])
print(avgsalaries)

#graphing this 
ax = sns.lineplot(x='Location', y='mean'
             ,data=avgsalaries)
plt.xticks(rotation=45)
ax.yaxis.set_major_formatter('${x:1.0f}')
plt.title('Average salary by location')
plt.ylabel('Mean Pay')
plt.xlabel('Location')

topindex["Location"] = topindex['City/region'].astype(str) +","+ topindex["State"]
first_column = topindex.pop('Location')
topindex.insert(1, 'Location', first_column)
topindex = topindex.drop(['City/region'],axis=1)
topindex = topindex.drop(['State'],axis=1)
topindex = topindex.drop(['Country'],axis=1)
topindex = topindex.drop(['Rank'],axis=1)
#merging these dataframes 
salaries_index = avgsalaries.merge(topindex, how = 'inner', on = ['Location'])
print(salaries_index)
#cross refrencing the results of the above code with salaries to find my top five places to live
# my top 5 are: 5.) Charlotte, NC, 4.) Dallas, TX, 3.) Raleigh, NC, 2.) Columbus, OH and #1.) is Houston, TX 
# Salaries_index has the top 10

FileNotFoundError: [Errno 2] No such file or directory: 'Python Project/cost_of_living.csv'